# Create a forge and execute an evolution cycle
In this notebook, we will go through the very few steps needed to run a forge cycle for a given budget. 
Before running this notebook, you must set up a few things:

**Your access to LLM's providers:**

To try Ebiose, you must have access to at least one LLM provider. To provide the credentials, copy-paste the [`model_endpoints_template.yml`](./../model_endpoints_template.yml) file, rename it as `model_endpoints.yml` and fill it with your own credentials.

**Add the root directory to your Python path**

Depending on your settings, you may need to add the root of the repository to your `PYTHONPATH` environment variable. You may also use a `.env` file to do so. copy-paste the [`.env.template`](./../.env.template) file, rename it as `.env` and fill it with your own root directory.

**Use LangFuse for tracing**

Ebiose has chosen LangFuse to provide easy and free observability, through its self-hosted capability. Refer to [Langfuse official documentation](https://langfuse.com/self-hosting) to set it up. Once done, fill `LANGFUSE_SECRET_KEY`, `LANGFUSE_PUBLIC_KEY` and `LANGFUSE_HOST` in the `.env` file.

Other observability tools might be used but are not configured yet.

**Load .env file**

To load the `.env` file, execute:

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

## Creating a basic forge

In ebiose, a **forge** is where custom agents are created to solve specific problems. The forge is the exclusive origin of new agents. Within each forge, architects agents orchestrate the creation and improvement of agents by reusing existing building blocks from the ecosystem.

To create a forge and run a cycle, you must provide the following:
- a description of the forge, which defines the problem that must be solved by generated agents;
- the expected format of the agent's input and output, defined as Pydantic models;
- an implementation of the `compute_fitness` abstract method that will be used by the forge to evaluate the generated agents.

Let's say we wich to generate agents specialized in solving math problems. The forge description could be:

In [2]:
forge_description = "Solving math word problems"

Next, we need to define the expected input and output formats of the generated agents. These formats are to be defined as Pydantic models. 

For instance, in our context of solving math problems, we want the agent input to be a string which will represent the math problem to be solved and the agent output to be composed of two fields:
- `solution` which will be the final solution to the math problem, given as an integer;
- `rationale` which will be the rationale behind the found solution.

The IO Pydantic models will thus be:

In [3]:
from pydantic import BaseModel

class AgentInput(BaseModel):
        math_problem: str

class AgentOutput(BaseModel):
    solution: int
    rationale: str

Lastly, we must provide a way of evaluating the generated agents through the implementation of the `compute_fitness` abstract method of `AgentForge` class. For the sake of demonstration, we will here return a random float between 0 and 1, so that we don't spend tokens at evaluation.

In [4]:
import random
random.seed(7)

from ebiose.core.agent import Agent
from ebiose.core.agent_forge import AgentForge

class BasicForge(AgentForge):
    async def compute_fitness(self, agent: Agent, compute_token_id: str, **kwargs: dict[str, any]) -> float:
        return random.random()

/Users/xabier/dev/ebiose-core/.venv/lib/python3.12/site-packages/pydantic/_internal/_generate_schema.py:502: UserWarning: <built-in function any> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(


We can now instantiate the forge with the provided elements:

In [5]:
forge = BasicForge(
    name="Basic forge",
    description=forge_description,
    agent_input_model=AgentInput,
    agent_output_model=AgentOutput,
    default_generated_agent_engine_type="langgraph_engine",
    default_model_endpoint_id="azure-gpt-4o-mini"
)

# Running a forge cycle

Once the forge is instantiated, we can start generating agents by running a **forge cycle**. 

To do so, you must define:
- a budget in dollars (the forge cycle will end once this budget is exhausted);
- optionally, a path in which created agents and fitness will be saved accross generations. 

> ⚠️ Note that we need to use `asyncio.run` to launch the forge cycle.

> 🚨 Before executing the following cell, check the amount of budget you have allocated!

> 💡 If you are using VSCode, install the [*Markdown Preview Mermaid Support* extension](https://marketplace.visualstudio.com/items?itemName=bierner.markdown-mermaid) to allow the display of the generated agent's graphs.

In [6]:
import asyncio
import nest_asyncio

from ebiose.core.evo_forging_cycle import EvoForgingCylceConfig
nest_asyncio.apply()

from pathlib import Path
from datetime import UTC, datetime

# the path where results will be saved
current_time = datetime.now(UTC).strftime("%Y-%m-%d_%H-%M-%S")
SAVE_PATH = Path(f"./../data/") / current_time
if not SAVE_PATH.exists():
    SAVE_PATH.mkdir(parents=True)

# budget for the forge cycle, in dollars
BUDGET = 0.01

cycle_config = EvoForgingCylceConfig(
    budget=BUDGET,
    n_agents_in_population=2,
    n_selected_agents_from_ecosystem=0,
    replacement_ratio=0.5,
    save_path=SAVE_PATH
)

final_agents, final_fitness = asyncio.run(
    forge.run_new_cycle(config=cycle_config)
)


Starting a new cycle for forge Basic forge
****** Initializing agents population ******
Creating 2 new agents with architect agents...
  0%|          | 0/2 [00:00<?, ?it/s]
Initializing structured output agent for model AgentOutput (1)
100%|██████████| 2/2 [00:24<00:00, 12.18s/it]
Agent initialization cost: 0.006423285000000001
Population initialized with 2 agents
Initialization of 2 agents took 0:00:24.409439
Budget left after initialization: 0.0035767149999999994 $
****** Running generation 0 ******
Evaluating current population of 2 agents...
100%|██████████| 2/2 [00:00<00:00, 3048.19it/s]
Agent agent-3f4dbecd-af15-443c-b6a2-005cdd66f48f fitness: 0.7943794815224912, cost: 0.0
Agent agent-a8c2abbc-f79b-4515-93b0-fd70d6d6945d fitness: 0.6989944337295713, cost: 0.0
Evaluation took 0:00:00.003617 for a total cost of 0.0 $


# Agent ID: agent-3f4dbecd-af15-443c-b6a2-005cdd66f48f
## Fitness: 0.7943794815224912
```mermaid 
graph LR
	Start_Node[start_node] --> Llm_Node_1(Problem Understanding LLM)
	Llm_Node_1(Problem Understanding LLM) --> Llm_Node_2(Solution Strategy LLM)
	Llm_Node_2(Solution Strategy LLM) --> End_Node[end_node]
 
``` 
## Prompts:
##### Shared context prompt
You are part of a collaborative AI model designed to solve math word problems. Each node plays a specific role in the problem-solving process. The 'Problem Understanding LLM' analyzes the word problem to extract key information such as numbers, operations, and relationships. The 'Solution Strategy LLM' develops a step-by-step approach to solve the problem based on the insights provided by the first LLM. Your task is to work seamlessly within this framework, ensuring that your output is clear, precise, and informative for the next node. Please pay attention to detail and use chain-of-thought reasoning to enhance your responses.
##### Problem Understanding LLM
Read the following math word problem carefully and extract all relevant information, including numerical values, operations, and relationships. Provide a comprehensive summary of the problem components, noting key elements that will influence the solution. Use chain-of-thought reasoning to ensure that your analysis captures all critical aspects needed for the next step. After summarizing, present the information clearly to facilitate the development of a solution strategy. Here is the math word problem: [INSERT MATH WORD PROBLEM HERE].
##### Solution Strategy LLM
Based on the information provided by the Problem Understanding LLM, develop a clear and logical step-by-step strategy to solve the math word problem. Begin by identifying the mathematical operations required and outline the methods to be used in the solution process. Include any assumptions or conditions that are relevant to the problem. Use reflective reasoning to evaluate the completeness of your strategy and ensure that it addresses all components extracted from the previous LLM. Your response should be structured and easy to follow for effective implementation. Here is the summarized information: [INSERT SUMMARY FROM LLMNode 1 HERE].

# Agent ID: agent-a8c2abbc-f79b-4515-93b0-fd70d6d6945d
## Fitness: 0.6989944337295713
```mermaid 
graph LR
	Start_Node[start_node] --> Math_Problem_Solver(Math Problem Solver)
	Math_Problem_Solver(Math Problem Solver) --> End_Node[end_node]
 
``` 
## Prompts:
##### Shared context prompt
You are part of an AI model designed to collaboratively solve math word problems through a structured problem-solving process. Your role is to analyze complex math word problems, extract relevant information, identify required mathematical operations, and derive solutions. You will communicate with other LLM nodes in the network, ensuring a smooth flow of information. Each LLM will contribute distinct capabilities to enhance the overall problem-solving capacity of the model, leveraging techniques such as chain-of-thought reasoning and self-reflection to arrive at accurate and efficient solutions.
##### Math Problem Solver
You are tasked with solving a math word problem. Begin by carefully reading the provided problem statement. Identify and extract all relevant information including quantities, relationships, and required operations (e.g., addition, subtraction, multiplication, division). 

Next, outline the steps you will take to solve the problem, employing a chain-of-thought approach to articulate your reasoning clearly. If you encounter any ambiguities or uncertainties in the problem, reflect on them critically, and consider alternative interpretations before proceeding. 

Finally, perform the calculations needed to arrive at the solution, and ensure to validate your answer by checking the logic and calculations you used. Once you have completed these steps, summarize your findings and provide the final answer to the problem. 

Make sure to communicate the solution in a clear and concise manner for the next node in the graph.



Saving current state to ../data/2025-03-05_00-27-19/generation=0
Starting crossover and mutation...

Initializing structured output agent for model AgentOutput (2)
Number of offsprings: 1/1
Crossover and mutation completed in 0:00:13.612888 for a total cost of 0.002049135 $
Generation 0 completed in 0:00:13.625793 with a total cost of 0.002049135 $
Budget left after first generation: 0.0015275799999999989 $
100%|██████████| 2/2 [00:00<00:00, 2587.48it/s]
Agent agent-3f4dbecd-af15-443c-b6a2-005cdd66f48f fitness: 0.7645708662128131, cost: 0.0
Agent agent-9a1d002a-db99-4fed-b138-e3aa2b6ec57d fitness: 0.573025940277384, cost: 0.0
Saving current state to ../data/2025-03-05_00-27-19/generation=1
Cycle completed in 0:00:38.059743 with a total cost of 0.008472420000000001 $
Budget left at final: 0.0015275799999999989 $
Returning 10 best agents


We can now display the best agents that have been returned as follows. Note that:
- all agents can be found in the `SAVE_PATH` directory if you defined one;
- here, the compute fitness only returns a random float, so the following displayed agents have not been truly evaluated. 

Go check [examples/math_forge/math_forge.py](./../examples/math_forge/math_forge.py) to see a fully implemeted forge with a non-random fitness evaluation function.

In [7]:
forge.display_results(final_agents, final_fitness)

# Agent ID: agent-3f4dbecd-af15-443c-b6a2-005cdd66f48f
## Fitness: 0.7645708662128131
```mermaid 
graph LR
	Start_Node[start_node] --> Llm_Node_1(Problem Understanding LLM)
	Llm_Node_1(Problem Understanding LLM) --> Llm_Node_2(Solution Strategy LLM)
	Llm_Node_2(Solution Strategy LLM) --> End_Node[end_node]
 
``` 
## Prompts:
##### Shared context prompt
You are part of a collaborative AI model designed to solve math word problems. Each node plays a specific role in the problem-solving process. The 'Problem Understanding LLM' analyzes the word problem to extract key information such as numbers, operations, and relationships. The 'Solution Strategy LLM' develops a step-by-step approach to solve the problem based on the insights provided by the first LLM. Your task is to work seamlessly within this framework, ensuring that your output is clear, precise, and informative for the next node. Please pay attention to detail and use chain-of-thought reasoning to enhance your responses.
##### Problem Understanding LLM
Read the following math word problem carefully and extract all relevant information, including numerical values, operations, and relationships. Provide a comprehensive summary of the problem components, noting key elements that will influence the solution. Use chain-of-thought reasoning to ensure that your analysis captures all critical aspects needed for the next step. After summarizing, present the information clearly to facilitate the development of a solution strategy. Here is the math word problem: [INSERT MATH WORD PROBLEM HERE].
##### Solution Strategy LLM
Based on the information provided by the Problem Understanding LLM, develop a clear and logical step-by-step strategy to solve the math word problem. Begin by identifying the mathematical operations required and outline the methods to be used in the solution process. Include any assumptions or conditions that are relevant to the problem. Use reflective reasoning to evaluate the completeness of your strategy and ensure that it addresses all components extracted from the previous LLM. Your response should be structured and easy to follow for effective implementation. Here is the summarized information: [INSERT SUMMARY FROM LLMNode 1 HERE].

# Agent ID: agent-9a1d002a-db99-4fed-b138-e3aa2b6ec57d
## Fitness: 0.573025940277384
```mermaid 
graph LR
	Start_Node[start_node] --> Llm_Node_1(Problem Understanding LLM)
	Llm_Node_1(Problem Understanding LLM) --> Llm_Node_2(Solution Strategy LLM)
	Llm_Node_2(Solution Strategy LLM) --> Llm_Node_3(Solution Reflection LLM)
	Llm_Node_3(Solution Reflection LLM) --> End_Node[end_node]
 
``` 
## Prompts:
##### Shared context prompt
You are part of a collaborative AI model designed to solve math word problems. Each node plays a specific role in the problem-solving process. The 'Problem Understanding LLM' analyzes the word problem to extract key information such as numbers, operations, and relationships. The 'Solution Strategy LLM' develops a step-by-step approach to solve the problem based on the insights provided by the first LLM. The 'Solution Reflection LLM' critiques and refines the strategy to enhance accuracy and completeness. Your task is to work seamlessly within this framework, ensuring that your output is clear, precise, and informative for the next node. Please pay attention to detail and use chain-of-thought reasoning to enhance your responses.
##### Problem Understanding LLM
Read the following math word problem carefully and extract all relevant information, including numerical values, operations, and relationships. Provide a comprehensive summary of the problem components, noting key elements that will influence the solution. Use chain-of-thought reasoning to ensure that your analysis captures all critical aspects needed for the next step. After summarizing, present the information clearly to facilitate the development of a solution strategy. Here is the math word problem: [INSERT MATH WORD PROBLEM HERE].
##### Solution Strategy LLM
Based on the information provided by the Problem Understanding LLM, develop a clear and logical step-by-step strategy to solve the math word problem. Begin by identifying the mathematical operations required and outline the methods to be used in the solution process. Include any assumptions or conditions that are relevant to the problem. Use reflective reasoning to evaluate the completeness of your strategy and ensure that it addresses all components extracted from the previous LLM. Your response should be structured and easy to follow for effective implementation. Here is the summarized information: [INSERT SUMMARY FROM LLMNode 1 HERE].
##### Solution Reflection LLM
Review the step-by-step strategy provided by the Solution Strategy LLM. Critique the strategy for potential errors or omissions, and suggest improvements or alternative approaches if necessary. Your critique should focus on ensuring that the solution is comprehensive and addresses all aspects of the math word problem. Here is the solution strategy: [INSERT STRATEGY FROM LLMNode 2 HERE].

